## Course 2:  Symbolic Execution
### **Exercise 1.** Practical Symbolic Execution using Manticore

The purpose of this exercise is to equip you with hand-on experience of applying *existing* symbolic execution tools to analyze smart contracts.
This exercise is based on [Manticore](https://github.com/trailofbits/manticore)—a symbolic execution engine developed in Python by Trail of Bits. *It is also inspired by the Manticore's own [tutorial](https://ethereum.org/en/developers/tutorials/how-to-use-manticore-to-find-smart-contract-bugs/).*
This Jupyter notebook uses Python 3.7.

#### **Installation**

Manticore (and Solidity compiler solc) can be installed using `pip`.

In [ ]:
!pip install "manticore[native]" solc-select
!solc-select install 0.5.11
!solc-select use 0.5.11

#### **Manticore Basics**

In this exercise, we will be using Python API provided by Manticore. Using the API commands, we can write a Python script controlling the symbolic execution process. First, the script should instantiate a new blockchain `m`:

In [2]:
from manticore.ethereum import ManticoreEVM

m = ManticoreEVM()

Now let's use Manticore to analyze the smart contract used as an example in the [theoretical write-up](https://github.com/baolean/formal-methods-curriculum/blob/course_2_symexec/courses/2_Approaches_Modeling_Verification/content/symbolic_execution.md). The next cell loads smart contract's source code into the `source_code` variable:

In [3]:
source_code = """
contract Exchange {   

    uint public balanceIn;
    uint public balanceOut;

    constructor() public {
        balanceIn = 100;
        balanceOut = 100;
    }

    function deposit(uint _in, uint approved, uint out) public {
        if (_in >= out && out <= balanceOut) {
            if (approved >= _in) {
                balanceIn += _in;
                approved -= _in;
            }
            
            balanceOut -= out;
        }

        assert(balanceIn + balanceOut >= 200);
   }
}
"""

To analyze a smart contract, it has to be deployed and, then, called via a transaction. To do so, we need to set up a regular (non-smart-contract) user account, which can be created using `m.create_account` command. It can be called with [several parameters](https://manticore.readthedocs.io/en/latest/evm.html?highlight=create_account#manticore.ethereum.ManticoreEVM.create_account) specyfing its balance, address, etc.

A smart contract defined in `source_code` can be, then, deployed using `m.solidity_create_contract` function. As parameters, we'll pass the smart contract code and the owner address we have just created. Other optional parameters can be found in the [documentation](https://manticore.readthedocs.io/en/latest/evm.html?highlight=create_account#manticore.ethereum.ManticoreEVM.create_contract). The script, then, becomes:

In [5]:
m = ManticoreEVM()

user_account = m.create_account(balance=100*10**18)
contract_account = m.solidity_create_contract(source_code, owner=user_account)

Manticore supports *raw* and *named* transactions. A *raw* transaction explores all smart contract functions—the transaction caller, data, address, or value can all be symbolic. However, in this example we will use *named* transactions which explore only one particular function, but it can do so with *symbolic* input arguments.

In Manticore API, symbolic variables (normally a uint256) are created using [`m.make_symbolic_value`](https://manticore.readthedocs.io/en/latest/evm.html?highlight=create_account#manticore.ethereum.ManticoreEVM.make_symbolic_value).
To call a function of a smart contract, run `contract_account.func_name(arguments)`.

**Task 1:** declare three symbolic variables representing `_in`, `_appr`, and `_out`, and call a `deposit(_in, _appr, _out)` function with these symbolic arguments.

In [6]:
m = ManticoreEVM()

user_account = m.create_account(balance=100*10**18)
contract_account = m.solidity_create_contract(source_code, owner=user_account)

# TODO: 
# (1) declare three symbolic variables representing `_in`, `_appr`, and `_out`
# (2) call a `deposit(_in, _appr, _out)` function with these symbolic arguments.

In [7]:
#@title Task 1 Solution 

m = ManticoreEVM()

user_account = m.create_account(balance=100*10**18)
contract_account = m.solidity_create_contract(source_code, owner=user_account)

symbolic_in = m.make_symbolic_value()
symbolic_out = m.make_symbolic_value()
symbolic_appr = m.make_symbolic_value()

contract_account.deposit(symbolic_in, symbolic_appr, symbolic_out);

Now let's perform the actual analysis of the smart contract and its `balanceIn + balanceOut >= 200` assertion.
As a symbolic execution tool, Manticore systematically analyses all possible execution paths. Each path ends in a certain blockchain state. The list of all final states can be accessed using `m.all_states`. In this exercise, we are interested in paths that end with a `THROW` (or `REVERT`) instruction, meaning that the execution was terminated—possibly, by a failed asserion. These states can be obtained via `m.terminated_states`.
Now, the concrete values corresponding to a state can be obtained using `m.generate_testcase(state, name="TestCaseName")`.
The results would be saved into a workplace directory `m.workspace`.

**Task 2:** obtain the list of states terminated states and generate a test case for each of them.

In [15]:
m = ManticoreEVM()

user_account = m.create_account(balance=100*10**18)
contract_account = m.solidity_create_contract(source_code, owner=user_account)

# TODO: 
# (1) declare three symbolic variables representing `_in`, `_appr`, and `_out`
# (2) call a `deposit(_in, _appr, _out)` function with these symbolic arguments.

# (3) obtain a list of terminated states
# (4) generate a test case for each of them

print(f'Assert violation found, results are in {m.workspace}')

Assert violation found, results are in /content/mcore_tncdv2ed


In [8]:
#@title Task 2 Solution

m = ManticoreEVM()

user_account = m.create_account(balance=100*10**18)
contract_account = m.solidity_create_contract(source_code, owner=user_account)

symbolic_in = m.make_symbolic_value()
symbolic_out = m.make_symbolic_value()
symbolic_appr = m.make_symbolic_value()

contract_account.deposit(symbolic_in, symbolic_appr, symbolic_out);

for state in m.terminated_states:
      m.generate_testcase(state, name="AssertViolated")
      print(f'Assert violation found, results are in {m.workspace}')


2022-06-17 12:28:37,435: [6702] m.c.plugin:WARNING: Caught will_solve in state None, but failed to capture its initialization
Assert violation found, results are in /content/mcore_lrw3_0m0
Assert violation found, results are in /content/mcore_lrw3_0m0


If we investigate the generated test cases, we will notice that two test cases are generated. After the manual symbolic execution described in the write-up, we are aware of *one* logical issue in the smart contract that might cause an assertion to fail. If we look at the concrete values generated by Manticore (they can be found in `AssertViolated_00000000.tx` and `AssertViolated_00000001.tx`), we'll notice that these values are, in fact, quite big, which might suggest that the failure is caused to integer underflow or overflow (it is). Now, for training purposes, let's assume that the caller can only use relatively small values as arguments when calling `deposit`—for example, less than or equal to 200.
*(Please **never** make such assumptions in practice)*.

This assumption can be expressed as a constraint on the symbolic values we declared earlier as `m.constrain(...)`. The constraint itself is an expression that can be constructed from a set of Operators available in Manticore. To import the Operators to your script, run `from manticore.core.smtlib import Operators`. Operators and their implementation can be found [here](https://github.com/trailofbits/manticore/blob/master/manticore/core/smtlib/operators.py). A *less-than-or-equal* Boolean expression corresponds to `Operators.ULE(operand1, operand2)` (`U` in `ULE` stands for `unsigned`—the details on that might/will be covered in next exercises).

**Task 3**: Add upper-bound value constraints (e.g., <= 200) on all three symbolic variables.

In [9]:
from manticore.core.smtlib import Operators

In [56]:
m = ManticoreEVM()

user_account = m.create_account(balance=100*10**18)
contract_account = m.solidity_create_contract(source_code, owner=user_account)

# TODO: 
# (1) declare three symbolic variables representing `_in`, `_appr`, and `_out`
# (5) add constraints on symbolic variables

# (2) call a `deposit(_in, _appr, _out)` function with these symbolic arguments.

# (3) obtain a list of terminated states
# (4) generate a test case for each of them

print(f'Assert violation found, results are in {m.workspace}')

Assert violation found, results are in /content/mcore_9wnu_s3p


In [10]:
#@title Task 3 Solution

m = ManticoreEVM()

user_account = m.create_account(balance=100*10**18)
contract_account = m.solidity_create_contract(source_code, owner=user_account)

symbolic_in = m.make_symbolic_value()
symbolic_out = m.make_symbolic_value()
symbolic_appr = m.make_symbolic_value()

m.constrain(Operators.ULE(symbolic_in, 200))
m.constrain(Operators.ULE(symbolic_out, 200))
m.constrain(Operators.ULE(symbolic_appr, 200))

contract_account.deposit(symbolic_in, symbolic_appr, symbolic_out);

for state in m.terminated_states:
      m.generate_testcase(state, name="AssertViolated")
      print(f'Assert violation found, results are in {m.workspace}')

2022-06-17 12:31:23,763: [6702] m.c.plugin:WARNING: Caught will_solve in state None, but failed to capture its initialization
Assert violation found, results are in /content/mcore_hszlm5r5


Although now we only generate a single expected test case, the assumptions we made on variable values are not realistic. Let's add overflow protection to the smart contract source code and remove the `assert` from the code (since it can overflow too). Instead, let's encode this requirement as a property in our Python script. Let's re-write the example smart contract source code by adding overflow protection to `balanceIn += _in;` and removing the `assert` statement.

In [11]:
source_code = """
contract Exchange {   

    uint public balanceIn;
    uint public balanceOut;

    constructor() public {
        balanceIn = 100;
        balanceOut = 100;
    }

    function deposit(uint _in, uint approved, uint out) public {
        if (_in >= out && out <= balanceOut) {
            if (approved >= _in) {
                balanceIn += _in;
                require(balanceIn >= _in);

                approved -= _in;
            }
            
            balanceOut -= out;
        }
   }
}
"""

Since now `assert` has been removed and overflowing executions revert, we need to analyze paths that executed successfuly: we can get them using `m.ready_states`.

We also need to define the property we'll be checking: that the sum of `balanceIn` and `balanceOut` should not decrease. In other words, we want to identify the \[vulnerable] states where this sum of balances before `deposit()` is *less* than the sum of balances after. We can identify these states by adding a corresponding constraint, or `condition`, by adding it to `m.generate_testcase(state, name="BugFound", only_if=condition)`.

However, we still need to formulate the property as an expression that Manticore understands. To do so, we need to obtain the values of `balanceIn` and `balanceOut` before and after the execution—they are returned by the following function calls: `contract_account.balanceIn()` and `contract_account.balanceOut()`.
<!-- These getters are automatically generated by Solidity. -->

**Task 4**: Instrument the script with four additional transactions that return smart contract balances (two before `deposit`, two after). Iterate through states that correspond to successful (not terminated) executions.

In [ ]:
#@title Task 4 Solution

m = ManticoreEVM()

user_account = m.create_account(balance=100*10**18)
contract_account = m.solidity_create_contract(source_code, owner=user_account)

symbolic_in = m.make_symbolic_value()
symbolic_out = m.make_symbolic_value()
symbolic_appr = m.make_symbolic_value()

# (1) Get values of balance_in and balance_out *before* deposit()

contract_account.deposit(symbolic_in, symbolic_appr, symbolic_out)

# (2) Get values of balance_in and balance_out *after* deposit()

for state in # (3) iterate over states that are *ready*
  print(state)

print(f'States info can be found in {m.workspace}')

In [12]:
#@title Task 4 Solution

m = ManticoreEVM()

user_account = m.create_account(balance=100*10**18)
contract_account = m.solidity_create_contract(source_code, owner=user_account)

symbolic_in = m.make_symbolic_value()
symbolic_out = m.make_symbolic_value()
symbolic_appr = m.make_symbolic_value()

contract_account.balanceIn()
contract_account.balanceOut()

contract_account.deposit(symbolic_in, symbolic_appr, symbolic_out)

contract_account.balanceIn()
contract_account.balanceOut()

for state in m.ready_states:
  print(state)

print(f'States info can be found in {m.workspace}')


<State object with id 8>
<State object with id 4>
<State object with id 1>
<State object with id 6>
States info can be found in /content/mcore_fjwnkh6f


To construct a property, we have to process the result returned by the `balanceIn()` and `balanceOut()` functions. The (serialized) data returned by a transaction *in a state* can be obtained using `state.platform.transactions[n].return_data` command, where `n` corresponds to the position of a transaction in a sequence of calls in the analysis script. For example, our script now has the following transactions and indices:

\[0] — m.solidity_create_contract

\[1] — contract_account.balanceIn()

\[2] — contract_account.balanceOut()

\[3] — contract_account.deposit(symbolic_in, symbolic_appr, symbolic_out)

...and so on.

As you may notice, the variable values we're interested in are returned by transactions 1, 2, 4, and 5. To be used in an expression, we need to deserialize the return data into an unsigned integer: this can be done by `ABI.deserialize("uint", return_data)`.
`ABI` needs to be imported prior to usage:
`from manticore.ethereum import ABI`.

After the balances before and after `deposit` are obtained, we can define a constraint that captures the vulnerable state criterion: the sum of balances *after* the function execution is *not smaller* than the sum *before*. This can be formulated using the *unsigned greater than* operator: `Operators.UGT(operand1, operand2)`.

**Task 5**: Process the data returned by `balanceIn()` and `balanceOut()`  calls. Use the obtained values to build a condition for test case generation, pass this condition to `m.generate_testcase` as an `only_if` argument.

In [14]:
from manticore.ethereum import ABI

In [ ]:
m = ManticoreEVM()

user_account = m.create_account(balance=100*10**18)
contract_account = m.solidity_create_contract(source_code, owner=user_account)

symbolic_in = m.make_symbolic_value()
symbolic_out = m.make_symbolic_value()
symbolic_appr = m.make_symbolic_value()

# (1) Get values of balance_in and balance_out *before* deposit()

contract_account.deposit(symbolic_in, symbolic_appr, symbolic_out)

# (2) Get values of balance_in and balance_out *after* deposit()

for state in m.ready_states:
    # (3) Get balance_in and balance_out *before* deposit() and sum them up
    # (4) Get balance_in and balance_out *after* deposit() and sum them up

    condition = # (5) define a condition that detects a vulnerable state
    
    # (6) pass the condition as only_if=... parameter in generate_testcase
    if m.generate_testcase(state, name="BugFound"):
        print(f'Bug found, results are in {m.workspace}')

In [15]:
#@title Task 5 Solution

m = ManticoreEVM()

user_account = m.create_account(balance=100*10**18)
contract_account = m.solidity_create_contract(source_code, owner=user_account)

symbolic_in = m.make_symbolic_value()
symbolic_out = m.make_symbolic_value()
symbolic_appr = m.make_symbolic_value()

contract_account.balanceIn()
contract_account.balanceOut()

contract_account.deposit(symbolic_in, symbolic_appr, symbolic_out)

contract_account.balanceIn()
contract_account.balanceOut()

for state in m.ready_states:
    in_balance_before = state.platform.transactions[1].return_data
    in_balance_before = ABI.deserialize("uint", in_balance_before)

    out_balance_before = state.platform.transactions[2].return_data
    out_balance_before = ABI.deserialize("uint", out_balance_before)

    in_balance_after = state.platform.transactions[4].return_data
    in_balance_after = ABI.deserialize("uint", in_balance_after)

    out_balance_after = state.platform.transactions[5].return_data
    out_balance_after = ABI.deserialize("uint", out_balance_after)

    sum_before = in_balance_before + out_balance_before
    sum_after = in_balance_after + out_balance_after

    condition = Operators.UGT(sum_before, sum_after)

    if m.generate_testcase(state, name="BugFound", only_if=condition):
        print(f'Bug found, results are in {m.workspace}')

2022-06-17 12:37:00,977: [6702] m.c.plugin:WARNING: Caught will_solve in state None, but failed to capture its initialization
Bug found, results are in /content/mcore_j7rcn9rr
Bug found, results are in /content/mcore_j7rcn9rr


We can, however, notice that two test cases are generated. One of them is, again, is attributed to integer overflow that occurs when we sum `balance_in` and `balance_out` before (and after) the execution. Since these operations are not part of the analyzed smart contract, let's add an additional "no-overflow" constraint to the test case generation condition.
Our final condition will consist of two parts: *the sum of balances decreased AND no overflow occured during the (balance_in + balance_out) operation*.
To implement AND, use `Operators.AND`. In the absence of integer overflow, the following condition should hold: *the result of a sum is greater than or equal (`Operators.UGT`) to the first operand*.

(Final) **Task 6** Integrate the no-overflow-on-a-sum constraint into a condition that is used to detect vulnerable state. Analyze the resulting test case, check if it matches the result of the manual analysis performed in the write-up.

In [ ]:
m = ManticoreEVM()

user_account = m.create_account(balance=100*10**18)
contract_account = m.solidity_create_contract(source_code, owner=user_account)

symbolic_in = m.make_symbolic_value()
symbolic_out = m.make_symbolic_value()
symbolic_appr = m.make_symbolic_value()

# (1) Get values of balance_in and balance_out *before* deposit()

contract_account.deposit(symbolic_in, symbolic_appr, symbolic_out)

# (2) Get values of balance_in and balance_out *after* deposit()

for state in m.ready_states:
    # (3) Get balance_in and balance_out *before* deposit() and sum them up
    # (4) Get balance_in and balance_out *after* deposit() and sum them up

    # (5) Define a constraint that ensures that no overflow
    #      occurs during balances-before sum-up 
    # (6) Define a constraint that ensures that no overflow
    #      occurs during balances-after sum-up 


    condition = # (7) define a condition includes no-overflow
                #   and property violation constraints
    
    # (8) pass the condition as only_if=... parameter in generate_testcase
    if m.generate_testcase(state, name="BugFound"):
        print(f'Bug found, results are in {m.workspace}')


In [16]:
#@title Task 6 Solution

m = ManticoreEVM()

user_account = m.create_account(balance=100*10**18)
contract_account = m.solidity_create_contract(source_code, owner=user_account)

symbolic_in = m.make_symbolic_value()
symbolic_out = m.make_symbolic_value()
symbolic_appr = m.make_symbolic_value()

contract_account.balanceIn()
contract_account.balanceOut()

contract_account.deposit(symbolic_in, symbolic_appr, symbolic_out)

contract_account.balanceIn()
contract_account.balanceOut()

for state in m.ready_states:
    in_balance_before = state.platform.transactions[1].return_data
    in_balance_before = ABI.deserialize("uint", in_balance_before)

    out_balance_before = state.platform.transactions[2].return_data
    out_balance_before = ABI.deserialize("uint", out_balance_before)

    in_balance_after = state.platform.transactions[4].return_data
    in_balance_after = ABI.deserialize("uint", in_balance_after)

    out_balance_after = state.platform.transactions[5].return_data
    out_balance_after = ABI.deserialize("uint", out_balance_after)

    sum_before = in_balance_before + out_balance_before
    sum_after = in_balance_after + out_balance_after
    
    no_before_overflow = Operators.UGE(sum_before, in_balance_before)
    no_after_overflow = Operators.UGE(sum_after, in_balance_after)
    no_sum_overflow = Operators.AND(no_before_overflow, no_after_overflow)

    condition = Operators.AND(Operators.UGT(sum_before, sum_after),
                                no_sum_overflow)

    if m.generate_testcase(state, name="BugFound", only_if=condition):
        print(f'Bug found, results are in {m.workspace}')

2022-06-17 12:38:39,371: [6702] m.c.plugin:WARNING: Caught will_solve in state None, but failed to capture its initialization
Bug found, results are in /content/mcore_lx0x9szd
